In [1]:
import findspark
findspark.init()
findspark.find()

'c:\\Users\\user\\miniconda3\\lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, col, lit
from datetime import datetime
import logging

In [3]:
spark = SparkSession.builder\
        .appName("Silver_Layer_Ingesion")\
        .enableHiveSupport()\
        .getOrCreate()

In [9]:
df = spark.sql("SELECT * FROM gfs_customers")

In [10]:
df.show()

+-----------+--------------------+--------------------+--------------------+---------------+-------------+------+-------------+---------+----+
|Customer_ID|                Name|               Email|           Telephone|           City|      Country|Gender|Date_Of_Birth|Job_Title|year|
+-----------+--------------------+--------------------+--------------------+---------------+-------------+------+-------------+---------+----+
|    1571768|     António Andrade|antónio.andrade@f...|   (351) 921 848 592|          Porto|     Portugal|     M|   2006-12-09|     NULL|2006|
|       2676|       Natalie Adams|natalie.adams@fak...|001-711-886-2633x...|       New York|United States|     F|   2006-05-17|     NULL|2006|
|    1573609|Andreia de Rodrigues|andreia.de.rodrig...|     (351) 931511410|          Porto|     Portugal|     F|   2006-12-02|     NULL|2006|
|       6167| Jonathon Becker DDS|jonathon.becker.d...|        900-645-2514|       New York|United States|     M|   2006-08-04|     NULL|2006|

In [ ]:
df.write \
  .format("jdbc") \
  .option("url", "jdbc:sqlserver://<sql-server>:1433;database=datawarehouse") \
  .option("dbtable", "LocalSQLServerTable") \
  .option("user", "<username>") \
  .option("password", "<password>") \
  .save()

In [16]:
spark.sql("CREATE DATABASE datawarehouse")

DataFrame[]

In [4]:
spark.sql("use datawarehouse")

DataFrame[]

In [5]:
spark.sql("create schema stage")
spark.sql("create schema gold")

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `stage` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [13]:
spark.sql("use stage").show()

++
||
++
++



In [15]:
spark.sql("CREATE TABLE IF NOT EXISTS stage.t_gfs_customers AS SELECT * FROM default.gfs_customers WHERE 1=0;")

DataFrame[]

In [16]:
spark.sql("INSERT INTO stage.t_gfs_customers SELECT * FROM default.gfs_customers;")

DataFrame[]

In [19]:
spark.sql("""
          select * from stage.t_gfs_customers
""").show()

+-----------+--------------------+--------------------+--------------------+---------------+-------------+------+-------------+---------+----+
|Customer_ID|                Name|               Email|           Telephone|           City|      Country|Gender|Date_Of_Birth|Job_Title|year|
+-----------+--------------------+--------------------+--------------------+---------------+-------------+------+-------------+---------+----+
|    1571768|     António Andrade|antónio.andrade@f...|   (351) 921 848 592|          Porto|     Portugal|     M|   2006-12-09|     NULL|2006|
|       2676|       Natalie Adams|natalie.adams@fak...|001-711-886-2633x...|       New York|United States|     F|   2006-05-17|     NULL|2006|
|    1573609|Andreia de Rodrigues|andreia.de.rodrig...|     (351) 931511410|          Porto|     Portugal|     F|   2006-12-02|     NULL|2006|
|       6167| Jonathon Becker DDS|jonathon.becker.d...|        900-645-2514|       New York|United States|     M|   2006-08-04|     NULL|2006|

In [20]:
spark.stop()